In [6]:
import os
from os.path import join as opj
import torch
import segmentation_models_pytorch as smp
from loguru import logger
import pandas as pd

seed = 42
name = 'Unet-resnext50_32x4d-512x512'
model_name = 'Unet'
backbone_name = 'resnext50_32x4d'
class_output = 1
data_folder = 'blood-vessel-segmentation'
train_folder = 'train'
test_folder = 'test'
weight_folder = 'weight'
# img_size  = [512, 512]

data_path = opj(os.getcwd(), data_folder)
train_path = opj(data_path, train_folder)
test_path = opj(data_path, test_folder)
weight_path = opj(os.getcwd(), weight_folder)

train_method = 2
debug = False
train_bs = 8
valid_bs = 8
img_size = [512, 512]
epochs = 30
n_accumulate = max(1, 64//train_bs)
lr = 1e-2
scheduler = 'CosineAnnealingLR'
min_lr = 5e-6
T_max = int((2279+1397+2217+1035)/(train_bs*n_accumulate)*epochs)+50
T_0 = 25
warmup_epochs = 5
wd = 1e-6
n_fold = 5
num_classes = 1

gt_df = opj(os.getcwd(), 'gt.csv')
train_groups = ["kidney_1_dense", "kidney_1_voi", "kidney_2", "kidney_3_sparse"]
valid_groups = ["kidney_3_dense"]
loss_func     = "MultiLoss_2"
thresh = 0.6

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def build_model(backbone, num_classes, device):
    model = smp.Unet(
        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_name=backbone,
        # use `imagenet` pre-trained weights for encoder initialization
        encoder_weights='imagenet',
        # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        in_channels=3,
        # model output channels (number of classes in your dataset)
        classes=num_classes,
        activation=None,
    )

    model.to(device)
    return model

In [4]:
model = build_model(backbone_name, 1, device)

In [ ]:
print(model.encoder.parameters)

In [16]:
print(dir(model.decoder))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save

In [15]:
print(model.decoder.get_parameter)

<bound method Module.get_parameter of UnetDecoder(
  (center): Identity()
  (blocks): ModuleList(
    (0): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(3072, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention1): Attention(
        (attention): Identity()
      )
      (conv2): Conv2dReLU(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention2): Attention(
        (attention): Identity()
      )
    )
    (1): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(768, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_runni

In [4]:
df = pd.read_csv(gt_df)

In [7]:
logger.info('len of kidney_1_dense: {}'.format(len(df.loc[df['group'] == 'kidney_1_dense'])))
logger.info('len of kidney_1_voi: {}'.format(len(df.loc[df['group'] == 'kidney_1_voi'])))
logger.info('len of kidney_2: {}'.format(len(df.loc[df['group'] == 'kidney_2'])))
logger.info('len of kidney_3_dense: {}'.format(len(df.loc[df['group'] == 'kidney_3_dense'])))
logger.info('len of kidney_3_sparse: {}'.format(len(df.loc[df['group'] == 'kidney_3_sparse'])))

2023-11-30 19:24:25.088 | INFO     | __main__:<module>:1 - len of kidney_1_dense: 2279
2023-11-30 19:24:25.089 | INFO     | __main__:<module>:2 - len of kidney_1_voi: 1397
2023-11-30 19:24:25.091 | INFO     | __main__:<module>:3 - len of kidney_2: 2217
2023-11-30 19:24:25.092 | INFO     | __main__:<module>:4 - len of kidney_3_dense: 501
2023-11-30 19:24:25.093 | INFO     | __main__:<module>:5 - len of kidney_3_sparse: 1035


In [6]:
imarray.shape

(1303, 912)

In [ ]:
img_size  = [512, 512]
data_transforms = {
    "train": A.Compose(
        [
            A.Resize(
                *img_size, interpolation=cv2.INTER_NEAREST
            ),
            A.HorizontalFlip(p=0.5),
        ],
        p=1.0
    ),
    "valid": A.Compose(
        [
            A.Resize(
                *img_size, interpolation=cv2.INTER_NEAREST
            ),
        ],
        p=1.0
    )
}

In [1]:
import timm

/home/tamdiep/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import segmentation_models_pytorch as smp
import torch

model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/tamdiep/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:05<00:00, 19.3MB/s]


In [ ]:
model.to(torch.device('cuda'))

In [8]:
!nvidia-smi

Wed Nov 15 16:10:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 40%   39C    P2    33W / 130W |    691MiB /  8192MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = np.tile(img[...,None], [1, 1, 3]) # gray to rgb
    img = img.astype('float32') # original is uint16
    mx = np.max(img)
    if mx:
        img /= mx # scale image to [0, 1]
    return img

def load_msk(path):
    msk = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    msk = msk.astype('float32')
    msk /= 255.0
    return msk